In [ ]:
!pip install transformers
!pip install qdrant-client

In [3]:
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import torch.nn as nn
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [5]:
import pandas as pd

def read_column_from_csv(file_path, column_name):
    df = pd.read_csv(file_path)
    return df[column_name].tolist()


In [7]:
food = read_column_from_csv("food_nutrients.csv", "Food")
calories = read_column_from_csv("food_nutrients.csv", "Calories (kcal)")
proteins = read_column_from_csv("food_nutrients.csv", "Proteins (g)")
fats = read_column_from_csv("food_nutrients.csv", "Fats (g)")
carbohydrates = read_column_from_csv("food_nutrients.csv", "Carbohydrates (g)")
sodium = read_column_from_csv("food_nutrients.csv", "Sodium (mg)")
cholesterol = read_column_from_csv("food_nutrients.csv", "Cholesterol (mg)")

In [8]:
text_embeddings = []

for i in range(0,len(food)):
  text_inputs = processor(text=[food[i]], return_tensors="pt", padding=True).to(device)
  text_emb = model.get_text_features(**text_inputs)
  text_embeddings.append(text_emb[0].squeeze().tolist())

In [9]:
len(text_embeddings[0])


512

In [10]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams


qdrant_client = QdrantClient(
    url="QDRANT_URL", #REPACE THIS WITH YOUR QDRANT URL
    api_key="API_KEY", #REPACE THIS WITH YOUR QDRANT URL
)


In [11]:

qdrant_client.recreate_collection(
    collection_name="food_nutrition_data",
    vectors_config=VectorParams(size=len(text_embeddings[0]), distance=Distance.COSINE),

)

<ipython-input-11-908037c3650e>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [12]:
payload = [
    {"food" : food[i],
     "calories" : calories[i],
     "proteins" : proteins[i],
     "fats" : fats[i],
     "carbohydrates" : carbohydrates[i],
     "sodium" : sodium[i],
     "cholestrol" : cholesterol[i]}
    for i in range(len(food))
]

qdrant_client.upsert(
    collection_name="food_nutrition_data",
    points=[
        {
            "id": i,
            "vector": text_embeddings[i],
            "payload": payload[i]
        }
        for i in range(len(food))
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)